In [15]:
import csv
import pandas
import numpy
import random
from CWHashingFamily import CWHashingFamily

import sys
reload(sys)
sys.setdefaultencoding('utf-8')

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [16]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv",minPartitions=None, use_unicode=False).mapPartitions(lambda x: csv.reader(x))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (int(line[ID]), shingles(line[TEXT],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] ))



In [17]:
from LSH import LSH
from Record import Record
import time

lshTrain = LSH(R,B)
#ESTO NO ANDA

trainRDD = trainRDD.take(300)

#def add_record(lshObj, line1, line2):
#    lshObj.add( Record(line1) )
#    return line2

#trainList = trainRDD.take(100)

#testRDD = sc.parallelize(trainList)

#trainRDD = trainRDD.reduce(lambda line1, line2: add_record(lshTrain, line1, line2) )

#ESTO ANDA:

start_time = time.time()

for i in xrange(len(trainRDD)):
    record = Record((trainRDD[i][0], trainRDD[i][1]) )
    lshTrain.add(record)
    
shinglesText = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
test = (492346,shingles(shinglesText,SHINGLES_LEN))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B) ])
testRecord = Record(test)

print lshTrain.getAllSimilarRecords(testRecord)

print("--- %s seconds ---" % (time.time() - start_time))


In [14]:
len(trainRDD)

300